In [ ]:
# TODO: update to the new data format, which simplifies things!
# TODO: update this from notebook to semi-interactive script

import csv

# data = list(csv.DictReader(open('data/exp2-annotated-processed.csv', 'r')))
data = list(csv.DictReader(open('data/exp2-all_raw_data.csv', 'r')))

print(len(data), data[0].keys())

In [ ]:

for line in data:
    x = line["nationality_parent"]
    line["nationality_parent_group"] = (
        "North American" if x in {"American", "Canadian"} else
        "South American" if x in {"Mexican", "Brazilian"} else
        "European" if x in {"British", "German", "French", "Italian", "Russian"} else
        "Middle Eastern" if x in {"Armenian", "Afghan", "Azerbaijani", "Egyptian", "Iranian", "Iraqi"} else
        "African" if x in {"Ethiopian", "Kenyan", "Malian", "Nigerian", "South African", "Sudanese"} else
        "Asian" if x in {"Chinese", "Filipino", "Indian", "Indonesian", "Japanese", "Sri Lankan", "Tajik", "Thai", "Vietnamese"} else
        "Other"
    )
    line["nationality_parent_developed"] = (
        "Developed" if x in {"American", "British", "Canadian", "French", "German", "Italian", "Japanese", "Russian"} else
        "Developing" if x in {"Afghan", "Armenian", "Azerbaijani", "Brazilian", "Chinese", "Egyptian", "Ethiopian", "Filipino", "Indian", "Indonesian", "Iranian", "Iraqi", "Kenyan", "Malian", "Mexican", "Nigerian", "South African", "Sri Lankan", "Sudanese", "Tajik", "Thai", "Vietnamese"} else
        "Other"
    )

In [ ]:
{d["model"] for d in data}

In [ ]:
data[-1]["protagonist_attributes_processed"]

In [ ]:
import sklearn.feature_extraction.text
import sklearn.linear_model
import sklearn.model_selection
import numpy as np

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(
    max_features=100,
    # these stop words include he/she/they/etc
    stop_words='english',
)
data_x = vectorizer.fit_transform([d['story_text_cleaned'] for d in data])
data_y = [d['gender_of_child'] for d in data]

data_x_train, data_x_test, data_y_train, data_y_test = sklearn.model_selection.train_test_split(data_x, data_y, test_size=0.1, random_state=0)

model = sklearn.linear_model.LogisticRegression()
model.fit(data_x_train, data_y_train)
print(f"Accuracy: {model.score(data_x_test, data_y_test):.1%}")

# get most important features from vectorizer and model
features = vectorizer.get_feature_names_out()

for target_i, target in enumerate(model.classes_):
    # top_indicies = np.argsort(np.abs(model.coef_[target_i]))[-10:][::-1]
    top_indicies = np.argsort(model.coef_[target_i])[-5:][::-1]
    texts = [f"& {model.coef_[target_i][i]:.1f} & {features[i]:>15}".replace("+-", "-") for i in top_indicies]
    print(
        "\\cmidrule{1-1}\n" +
        f"\\parbox[t]{{2mm}}{{\multirow{{5}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{target}}}}}}}",
        # f"To predict `{target}` we use the following:",
        end="\n",
    )
    print(
        *texts,
        sep="\\\\\n",
        end="\\\\\n",
    )